<a href="https://colab.research.google.com/github/henry-bokyum-kim/NNStudy/blob/master/GymDisplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)

import math
import glob
import io
import base64
from IPython.display import HTML

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400,900),)
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    for mp4 in mp4list:
      video = io.open(mp4, 'r+b').read()
      print(mp4)
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 300px;">
                                  <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                                  </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")

In [0]:
# default random move.

env = Monitor(gym.make("CartPole-v0"),"./video",force=True)
env.reset()
#env.render()

reward = 0.0
count =0

while True:
  action = count%2
  obs, rew, done, _ = env.step(action)
  reward += rew
  count +=1

  if done:
    break
env.close()

print("total Rew : %.4f"%reward)
show_video()

In [0]:
#basic NN
import gym
from collections import namedtuple
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

HIDDEN_SIZE=128
BATCH_SIZE=32
PRUNE=50

class Net(nn.Module):
  def __init__(self, obs_size, hidden_size, n_actions):
    super(Net, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(obs_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,n_actions)
    )
  def forward(self, x):
    return self.net(x)

episode = namedtuple('Episode', field_names=['reward','steps'])
episodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

def iterate_batches(env, net, batch_size):
  batch=[]
  episode_reward = 0.0
  episode_steps = []
  obs = env.reset()
  sm = nn.Softmax(dim=1)
  while True:
    obs_v = torch.FloatTensor([obs])
    act_probs_v = sm(net(obs_v))
    act_probs = act_probs_v.data.numpy()[0]
    action = np.random.choice(len(act_probs), p=act_probs)

    next_obs, reward, done, _ = env.step(action)
    episode_reward += reward
    episode_steps.append(episodeStep(observation = obs, action=action))
    if done:
      batch.append(episode(reward=episode_reward, steps=episode_steps))
      episode_reward = 0.0
      episode_steps = []
      next_obs = env.reset()
      if len(batch) == batch_size:
        yield batch
        batch = []
    obs=next_obs

def filter_batch(batch, percentile):
  rewards = list(map(lambda s:s.reward, batch))
  reward_bound = np.percentile(rewards, percentile)
  reward_mean = float(np.mean(rewards))

  train_obs = []
  train_act = []

  for example in batch:
    if example.reward < reward_bound:
      continue
    train_obs.extend(map(lambda step: step.observation, example.steps))
    train_act.extend(map(lambda step: step.action, example.steps))

  train_obs_v = torch.FloatTensor(train_obs)
  train_act_v = torch.LongTensor(train_act)
  return train_obs_v, train_act_v, reward_bound, reward_mean

if __name__ == "__main__":
  env = gym.wrappers.Monitor(gym.make("CartPole-v1"), "./video", force=True, video_callable=lambda cnt: cnt%100==0)
  obs_size = env.observation_space.shape[0]
  n_actions = env.action_space.n

  net = Net(obs_size, HIDDEN_SIZE, n_actions)
  objective = nn.CrossEntropyLoss()
  optimizer = optim.Adam(params=net.parameters(), lr=0.01)

  for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
      obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PRUNE)
      optimizer.zero_grad()
      action_scores_v = net(obs_v)
      loss_v = objective(action_scores_v, acts_v)
      loss_v.backward()
      optimizer.step()
      print("%d: loss=%.3f, reward_mean=%.2f, reward_bound=%.2f" % (iter_no, loss_v.item(), reward_m, reward_b))
      #if iter_no%20 == 0:
        #show_video()
      if reward_m > 490:
          print("Solved!")
          #show_video()
          break

In [0]:
show_video()